# 🔬 QuiverQuant Data Exploration Notebook

**Objectif:** Tester et explorer les données QuiverQuant petit à petit pour comprendre les formats, la qualité et la structure.

**Date:** 31 Décembre 2025  
**Token:** bibep  

---

## 📋 Plan d'exploration

1. ✅ Import des libraries
2. ✅ Connexion à QuiverQuant
3. ✅ Congressional Trading - Tous les trades
4. ✅ Congressional Trading - Par ticker spécifique
5. ✅ Senate vs House Trading
6. ✅ Insider Trading
7. ✅ WallStreetBets Sentiment
8. ✅ Analyse comparative
9. ✅ Nettoyage et standardisation
10. ✅ Export et intégration

## 1️⃣ Import des Libraries

In [ ]:
import sys
import os
from pathlib import Path

# Déterminer le répertoire racine du projet
notebook_dir = Path(r'c:\project\n8n-dashboard-v4\notebooks')
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

# Ajouter les chemins importants
sys.path.insert(0, str(project_root / 'prod'))
sys.path.insert(0, str(project_root / 'services'))

print(f"📂 Project root: {project_root}")
print(f"📂 Paths ajoutés: {sys.path[:3]}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json

# ✅ IMPORT CONFIG - SINGLE SOURCE OF TRUTH
try:
    from config.companies_config import get_all_companies, COMPANIES_CONFIG
    print("✅ companies_config importé avec succès")
    
    # Afficher les tickers configurés
    companies = get_all_companies()
    TICKERS = [c['ticker'] for c in companies]
    print(f"   📊 Tickers configurés: {TICKERS}")
    print(f"   📊 Total: {len(TICKERS)} compagnies")
except ImportError as e:
    print(f"⚠️ Erreur import companies_config: {e}")
    TICKERS = None

# Importer le client QuiverQuant depuis la structure correcte
try:
    from quiverquant.quiverquant_client import QuiverQuantClient
    from quiverquant.config import QUIVERQUANT_TOKEN
    print("✅ QuiverQuant modules importés avec succès")
except ImportError as e:
    print(f"⚠️ Erreur import QuiverQuant: {e}")
    print("   Vérifier que les modules existent dans la branche _political_trading")
    QUIVERQUANT_TOKEN = None

print("✅ Libraries importées")
print(f"📅 Date/heure: {datetime.now()}")
print(f"🔐 Token disponible: {'✅' if QUIVERQUANT_TOKEN else '❌ (module non trouvé)'}")


📂 Project root: c:\project\n8n-dashboard-v4
📂 Paths ajoutés: ['c:\\project\\n8n-dashboard-v4\\services', 'c:\\project\\n8n-dashboard-v4\\prod', 'c:\\project\\n8n-dashboard-v4']
⚠️ Erreur import QuiverQuant: No module named 'quiverquant'
   Vérifier que les modules existent dans la branche _political_trading
✅ Libraries importées
📅 Date/heure: 2026-01-06 11:05:43.998917
🔐 Token disponible: ❌ (module non trouvé)


## 2️⃣ Connexion à QuiverQuant & Première Requête

In [ ]:
# Initialiser le client
print("🔌 Initialisation du client QuiverQuant...")
client = QuiverQuantClient(QUIVERQUANT_TOKEN)
print("✅ Client prêt!")

# ✅ STEP 1: AFFICHER LA CONFIG
print("\n" + "="*70)
print("📋 CONFIGURATION: Tickers à analyser")
print("="*70)
print(f"\n📍 Source: prod/config/companies_config.py")
print(f"📊 Tickers ({len(TICKERS)}):")
for i, ticker in enumerate(TICKERS, 1):
    company = next((c for c in companies if c['ticker'] == ticker), None)
    if company:
        print(f"   {i:2d}. {ticker:6s} - {company['name']:30s} ({company['sector']})")

# Première requête: Congressional Trading
print("\n" + "="*70)
print("📡 STEP 1: Congressional Trading (Tous les trades récents)")
print("="*70)

try:
    df_congress = client.congress_trading()
    print(f"✅ Succès! {len(df_congress)} trades récupérés")
    print(f"\n📊 Shape: {df_congress.shape}")
    print(f"\n📋 Colonnes: {list(df_congress.columns)}")
    print(f"\n🔍 Premiers 3 trades:")
    print(df_congress.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")


🔌 Initialisation du client QuiverQuant...


NameError: name 'QuiverQuantClient' is not defined

## 3️⃣ Exploration Détaillée - Congressional Trading

In [ ]:
print("📊 STEP 3: ANALYSE CONGRESSIONAL TRADING - Aligné avec Config")
print("="*70)

# ✅ Toujours utiliser df_congress_filtered (déjà filtré par tickers configurés)
df_analysis = df_congress_filtered.copy()

print(f"\n📍 Source: companies_config.py ({len(TICKERS)} tickers)")
print(f"📊 Données: Congressional Trading (filtered)\n")

# Info générales
print("1️⃣ INFO GÉNÉRALES:")
print(f"   Nombre de trades: {len(df_analysis)}")
print(f"   Nombre de tickers configurés avec données: {df_analysis['Ticker'].nunique()}/{len(TICKERS)}")
if 'TransactionDate' in df_analysis.columns:
    df_analysis_dt = df_analysis.copy()
    df_analysis_dt['TransactionDate'] = pd.to_datetime(df_analysis_dt['TransactionDate'])
    print(f"   Date min: {df_analysis_dt['TransactionDate'].min()}")
    print(f"   Date max: {df_analysis_dt['TransactionDate'].max()}")

# Types de données
print("\n2️⃣ TYPES DE DONNÉES:")
print(df_analysis.dtypes)

# Valeurs manquantes
print("\n3️⃣ VALEURS MANQUANTES:")
missing = df_analysis.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✅ Aucune valeur manquante!")

# Statistiques descriptives
print("\n4️⃣ STATISTIQUES DESCRIPTIVES:")
print(df_analysis.describe())

# Politiciens uniques
if 'Representative' in df_analysis.columns:
    print(f"\n5️⃣ POLITICIENS UNIQUES: {df_analysis['Representative'].nunique()}")
    print("   Top 10:")
    print(df_analysis['Representative'].value_counts().head(10))

# Tickers (should be subset of TICKERS)
if 'Ticker' in df_analysis.columns:
    print(f"\n6️⃣ TICKERS DANS DONNÉES (from config):")
    tickers_in_data = df_analysis['Ticker'].unique()
    tickers_missing = set(TICKERS) - set(tickers_in_data)
    print(f"   ✅ Présents: {len(tickers_in_data)}")
    print(df_analysis['Ticker'].value_counts().sort_index())
    if tickers_missing:
        print(f"\n   ⚠️ Pas de données: {', '.join(sorted(tickers_missing))}")

# Types de transactions
if 'Transaction' in df_analysis.columns:
    print(f"\n7️⃣ TYPES DE TRANSACTIONS:")
    print(df_analysis['Transaction'].value_counts())


📊 ANALYSE CONGRESSIONAL TRADING

1️⃣ INFO GÉNÉRALES:
   Nombre de trades: 1000
   Date min: 2025-01-02 00:00:00
   Date max: 2025-12-16 00:00:00

2️⃣ TYPES DE DONNÉES:
Representative             object
BioGuideID                 object
ReportDate         datetime64[ns]
TransactionDate    datetime64[ns]
Ticker                     object
Transaction                object
Range                      object
House                      object
Amount                     object
Party                      object
last_modified              object
TickerType                 object
Description                object
ExcessReturn              float64
PriceChange               float64
SPYChange                 float64
dtype: object

3️⃣ VALEURS MANQUANTES:
Description     976
ExcessReturn     22
PriceChange      22
SPYChange        22
dtype: int64

4️⃣ STATISTIQUES DESCRIPTIVES:
                       ReportDate             TransactionDate  ExcessReturn  \
count                        1000            

## 4️⃣ Test Spécifique - Trades Tesla par Congress

In [ ]:
print("🔍 STEP 2: Filtrer Données par Tickers Configurés")
print("="*70)
print(f"📍 Source: companies_config.py → TICKERS list")
print(f"   Filtrer df_congress pour ne garder que les {len(TICKERS)} tickers configurés\n")

# Tickers de la config
print(f"✅ Tickers à analyser (from config): {TICKERS}\n")

# Filtrer df_congress pour tickers configurés
df_congress_filtered = df_congress[df_congress['Ticker'].isin(TICKERS)].copy()

print(f"📊 AVANT filtrage: {len(df_congress)} trades totaux")
print(f"📊 APRÈS filtrage: {len(df_congress_filtered)} trades pour {len(TICKERS)} tickers configurés")

# Analyser par ticker
print(f"\n📈 DISTRIBUTION PAR TICKER (configurés):")
ticker_counts = df_congress_filtered['Ticker'].value_counts().sort_index()
for ticker in TICKERS:
    count = ticker_counts.get(ticker, 0)
    company = next((c for c in companies if c['ticker'] == ticker), {})
    sector = company.get('sector', 'N/A')
    status = "✅" if count > 0 else "⚠️"
    print(f"   {status} {ticker:6s} ({sector:15s}): {count:3d} trades")

# Détails pour tickers avec données
print(f"\n📋 DÉTAILS DES TRADES PAR TICKER:")
for ticker in TICKERS:
    df_ticker = df_congress_filtered[df_congress_filtered['Ticker'] == ticker]
    
    if len(df_ticker) > 0:
        print(f"\n   {ticker}:")
        
        # Statistiques BUY/SELL
        if 'Transaction' in df_ticker.columns:
            buys = (df_ticker['Transaction'].str.contains('Purchase', case=False, na=False)).sum()
            sells = (df_ticker['Transaction'].str.contains('Sale', case=False, na=False)).sum()
            print(f"      📈 BUY: {buys} | SELL: {sells}")
            
        # Date range
        if 'TransactionDate' in df_ticker.columns:
            df_ticker_copy = df_ticker.copy()
            df_ticker_copy['TransactionDate'] = pd.to_datetime(df_ticker_copy['TransactionDate'])
            date_min = df_ticker_copy['TransactionDate'].min()
            date_max = df_ticker_copy['TransactionDate'].max()
            print(f"      📅 Date range: {date_min.date()} → {date_max.date()}")
        
        # Top politicians
        if 'Representative' in df_ticker.columns:
            top_rep = df_ticker['Representative'].value_counts().head(1)
            if len(top_rep) > 0:
                print(f"      👤 Top trader: {top_rep.index[0]} ({top_rep.values[0]} trades)")

print("\n" + "="*70)
print("✅ Filtrage terminé - Données alignées avec companies_config")


🔍 TEST TRADES PAR TICKER SPÉCIFIQUE
Note: L'endpoint par ticker retourne des erreurs JSON.
Solution: Filtrer le DataFrame complet récupéré précédemment


📊 Analyse de TSLA:
   ✅ 2 trades trouvés dans df_congress
   📋 Premiers trades:
      - Gilbert Cisneros: Purchase on 2025-11-18 00:00:00
      - Lisa Mcclain: Sale on 2025-10-31 00:00:00
   📈 BUY: 1 | SELL: 1
   📅 Date range: 2025-10-31 → 2025-11-18

📊 Analyse de NVDA:
   ✅ 15 trades trouvés dans df_congress
   📋 Premiers trades:
      - Dwight Evans: Purchase on 2025-11-21 00:00:00
      - Gilbert Cisneros: Purchase on 2025-11-18 00:00:00
      - Lisa Mcclain: Sale on 2025-10-31 00:00:00
   📈 BUY: 9 | SELL: 1
   📅 Date range: 2025-01-06 → 2025-11-21

📊 Analyse de MSFT:
   ✅ 26 trades trouvés dans df_congress
   📋 Premiers trades:
      - Dwight Evans: Sale on 2025-11-21 00:00:00
      - Cleo Fields: Purchase on 2025-12-11 00:00:00
      - Cleo Fields: Purchase on 2025-12-11 00:00:00
   📈 BUY: 12 | SELL: 2
   📅 Date range: 2025-01-03

## 5️⃣ Senate vs House Trading (Données Séparées)

In [ ]:
print("🏛️ STEP 4: SENATE vs HOUSE TRADING - Filtrés par Config")
print("="*70)
print(f"📍 Source: companies_config.py ({len(TICKERS)} tickers)\n")

# Senate
print("1️⃣ SENATE TRADING:")
try:
    df_senate = client.senate_trading()
    # ✅ Filtrer par tickers configurés
    df_senate_filtered = df_senate[df_senate['Ticker'].isin(TICKERS)].copy()
    print(f"   ✅ Total: {len(df_senate)} trades")
    print(f"   ✅ Filtrés (config tickers): {len(df_senate_filtered)} trades")
    
    if len(df_senate_filtered) > 0:
        print(f"   Colonnes: {list(df_senate_filtered.columns)}")
        print(f"   Tickers avec données: {df_senate_filtered['Ticker'].unique().tolist()}")
        print(f"\n   Premiers 3:")
        print(df_senate_filtered.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")
    df_senate_filtered = None

# House
print("\n2️⃣ HOUSE TRADING:")
try:
    df_house = client.house_trading()
    # ✅ Filtrer par tickers configurés
    df_house_filtered = df_house[df_house['Ticker'].isin(TICKERS)].copy()
    print(f"   ✅ Total: {len(df_house)} trades")
    print(f"   ✅ Filtrés (config tickers): {len(df_house_filtered)} trades")
    
    if len(df_house_filtered) > 0:
        print(f"   Colonnes: {list(df_house_filtered.columns)}")
        print(f"   Tickers avec données: {df_house_filtered['Ticker'].unique().tolist()}")
        print(f"\n   Premiers 3:")
        print(df_house_filtered.head(3))
except Exception as e:
    print(f"❌ Erreur: {e}")
    df_house_filtered = None

# Comparaison
print("\n3️⃣ COMPARAISON (après filtrage par config):")
senate_count = len(df_senate_filtered) if df_senate_filtered is not None else 0
house_count = len(df_house_filtered) if df_house_filtered is not None else 0
congress_count = len(df_congress_filtered)

print(f"   Senate:       {senate_count:4d} trades")
print(f"   House:        {house_count:4d} trades")
print(f"   Congress*:    {congress_count:4d} trades (combined)")
print(f"   (* from earlier retrieval, all {len(TICKERS)} tickers)")

print("\n4️⃣ BREAKDOWN PAR CHAMBER:")
for ticker in TICKERS:
    senate_trades = len(df_senate_filtered[df_senate_filtered['Ticker'] == ticker]) if df_senate_filtered is not None else 0
    house_trades = len(df_house_filtered[df_house_filtered['Ticker'] == ticker]) if df_house_filtered is not None else 0
    congress_trades = len(df_congress_filtered[df_congress_filtered['Ticker'] == ticker])
    
    if congress_trades > 0:
        print(f"   {ticker}: Senate={senate_trades:2d} | House={house_trades:2d} | Total={congress_trades:2d}")


🏛️ SENATE vs HOUSE TRADING

1️⃣ SENATE TRADING:
✅ 5000 trades du Sénat
   Colonnes: ['Senator', 'BioGuideID', 'Date', 'Ticker', 'Transaction', 'Range', 'Amount', 'last_modified']
   
   Premiers 3:
                     Senator BioGuideID       Date Ticker Transaction  \
0  A. Mitchell Jr. McConnell    M000355 2025-12-01    WFC    Purchase   
1         David H. McCormick    M001243 2025-11-28   BITB    Purchase   
2         David H. McCormick    M001243 2025-11-26   BITB    Purchase   

                Range   Amount last_modified  
0    $1,001 - $15,000   1001.0    2025-12-19  
1  $50,001 - $100,000  50001.0    2025-12-26  
2  $50,001 - $100,000  50001.0    2025-12-26  

2️⃣ HOUSE TRADING:
✅ 5000 trades du Sénat
   Colonnes: ['Senator', 'BioGuideID', 'Date', 'Ticker', 'Transaction', 'Range', 'Amount', 'last_modified']
   
   Premiers 3:
                     Senator BioGuideID       Date Ticker Transaction  \
0  A. Mitchell Jr. McConnell    M000355 2025-12-01    WFC    Purchase   
1    

## 6️⃣ Insider Trading

In [ ]:
print("👔 STEP 5: INSIDER TRADING - Filtrés par Config")
print("="*70)
print(f"📍 Source: companies_config.py ({len(TICKERS)} tickers)\n")

try:
    df_insiders = client.insiders()
    # ✅ Filtrer par tickers configurés
    df_insiders_filtered = df_insiders[df_insiders['Ticker'].isin(TICKERS)].copy()
    
    print(f"✅ Total insiders: {len(df_insiders)} trades")
    print(f"✅ Filtrés (config tickers): {len(df_insiders_filtered)} trades")
    
    print(f"\n📊 Shape: {df_insiders_filtered.shape}")
    print(f"\n📋 Colonnes: {list(df_insiders_filtered.columns)}")
    
    print(f"\n🔍 Premiers 5 insiders:")
    print(df_insiders_filtered.head(5))
    
    # Analyze
    if 'Ticker' in df_insiders_filtered.columns:
        print(f"\n📈 DISTRIBUTION PAR TICKER (configurés):")
        for ticker in TICKERS:
            count = len(df_insiders_filtered[df_insiders_filtered['Ticker'] == ticker])
            if count > 0:
                print(f"   {ticker}: {count} trades")
    
    if 'Relationship' in df_insiders_filtered.columns:
        print(f"\n👤 TYPES DE RELATIONSHIPS (configurés):")
        print(df_insiders_filtered['Relationship'].value_counts().head(10))
    
    # Transaction types
    if 'TransactionType' in df_insiders_filtered.columns:
        print(f"\n📊 TYPES DE TRANSACTIONS (insider):")
        print(df_insiders_filtered['TransactionType'].value_counts())
        
except Exception as e:
    print(f"❌ Erreur: {e}")
    df_insiders_filtered = None


👔 INSIDER TRADING DATA
📡 Fetching: https://api.quiverquant.com/beta/live/insiders
❌ Erreur: If using all scalar values, you must pass an index
❌ Erreur: If using all scalar values, you must pass an index


## 7️⃣ WallStreetBets Sentiment

In [ ]:
print("💬 STEP 6: WALLSTREETBETS SENTIMENT - Filtrés par Config")
print("="*70)
print(f"📍 Source: companies_config.py ({len(TICKERS)} tickers)\n")
print("⚠️ Note: Cet endpoint peut nécessiter un abonnement premium\n")

# Essayer différentes méthodes selon la documentation
methods = [
    ("Yesterday's data", lambda: client.wallstreetbets(yesterday=True)),
    ("Date range (last 7 days)", lambda: client.wallstreetbets(
        date_from=(datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d"),
        date_to=datetime.now().strftime("%Y-%m-%d")
    )),
    ("All data (défaut)", lambda: client.wallstreetbets()),
]

df_wsb = None
df_wsb_filtered = None
success = False

for method_name, method_func in methods:
    print(f"🔍 Tentative: {method_name}")
    try:
        df_wsb = method_func()
        if df_wsb is not None and len(df_wsb) > 0:
            # ✅ Filtrer par tickers configurés
            df_wsb_filtered = df_wsb[df_wsb['Ticker'].isin(TICKERS)].copy()
            print(f"   ✅ Succès! {len(df_wsb)} mentions trouvées")
            print(f"   ✅ Filtrés (config tickers): {len(df_wsb_filtered)} mentions")
            success = True
            break
        else:
            print(f"   ⚠️ Réponse vide")
    except Exception as e:
        error_msg = str(e)
        if "Upgrade your subscription" in error_msg:
            print(f"   ❌ Premium requis: {error_msg[:60]}")
        elif "If using all scalar values" in error_msg:
            print(f"   ❌ Format réponse invalide (possiblement vide ou premium)")
        else:
            print(f"   ❌ Échec: {error_msg[:60]}")

if success and df_wsb_filtered is not None and len(df_wsb_filtered) > 0:
    print(f"\n" + "="*70)
    print("📊 ANALYSE DES DONNÉES WSB (filtrées par config)")
    print("="*70)
    
    print(f"\n📊 Shape: {df_wsb_filtered.shape}")
    print(f"📋 Colonnes: {list(df_wsb_filtered.columns)}")
    
    print(f"\n🔍 Premiers 5 mentions:")
    print(df_wsb_filtered.head(5))
    
    # Analyze
    if 'Ticker' in df_wsb_filtered.columns:
        print(f"\n📈 TOP TICKERS MENTIONNÉS (configurés):")
        print(df_wsb_filtered['Ticker'].value_counts().sort_index())
        
    # Date range
    if 'Date' in df_wsb_filtered.columns:
        print(f"\n📅 DATE RANGE:")
        print(f"   Min: {df_wsb_filtered['Date'].min()}")
        print(f"   Max: {df_wsb_filtered['Date'].max()}")
    
    # Sentiment analysis (si disponible)
    if 'Sentiment' in df_wsb_filtered.columns:
        print(f"\n😊 SENTIMENT ANALYSIS (configurés):")
        print(df_wsb_filtered['Sentiment'].value_counts())
    
    # Mentions analysis (si disponible)
    if 'Mentions' in df_wsb_filtered.columns:
        print(f"\n📊 MENTIONS STATISTICS (configurés):")
        print(f"   Total mentions: {df_wsb_filtered['Mentions'].sum()}")
        print(f"   Average: {df_wsb_filtered['Mentions'].mean():.1f}")
        print(f"   Median: {df_wsb_filtered['Mentions'].median():.1f}")
else:
    print(f"\n" + "="*70)
    print("⚠️ ENDPOINT WALLSTREETBETS NON DISPONIBLE")
    print("="*70)
    print("""
Raisons possibles:
   ✓ Endpoint nécessite abonnement premium
   ✓ Réponse API vide ou format différent
   ✓ Données pas disponibles pour le compte actuel (bibep)

Solution:
   - Vérifier abonnement sur quiverquant.com
   - Contacter chris@quiverquant.com pour upgrade
   - Les endpoints Congressional/Senate/House fonctionnent correctement ✅

Alternative:
   - Utiliser reddit API directement
   - Scraper r/wallstreetbets manuellement
   - Focus sur political trading data (disponible)
    """)


💬 WALLSTREETBETS SENTIMENT
⚠️ Note: Cet endpoint peut nécessiter un abonnement premium

🔍 Tentative: Yesterday's data
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: Date range (last 7 days)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true&date_from=20251226&date_to=2026-01-02
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: Date range (last 7 days)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true&date_from=20251226&date_to=2026-01-02
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: All data (défaut)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true
   ❌ Format réponse invalide (possiblement vide ou premium)
🔍 Tentative: All data (défaut)
📡 Fetching: https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true
   ❌ Format réponse invalide (possiblement 

## 8️⃣ Analyse Comparative & Standardisation

In [ ]:
print("🔄 STEP 7: STANDARDISATION DES DONNÉES - Using Config")
print("="*70)
print(f"📍 Source: companies_config.py + Congressional Trading\n")

# Créer une version standardisée des congressional trades (déjà filtrés)
print("1️⃣ CONGRESSIONAL TRADES - NETTOYAGE & STANDARDISATION:\n")

df_congress_clean = df_congress_filtered.copy()

# Rename columns pour cohérence
column_mapping = {
    'Representative': 'politician_name',
    'TransactionDate': 'transaction_date',
    'ReportDate': 'report_date',
    'Ticker': 'ticker',
    'Transaction': 'transaction_type',
    'Amount': 'amount',
    'Chamber': 'chamber'
}

df_congress_clean = df_congress_clean.rename(columns={k: v for k, v in column_mapping.items() if k in df_congress_clean.columns})

# Add source
df_congress_clean['source'] = 'Congress (QuiverQuant)'
df_congress_clean['data_type'] = 'political_trade'
df_congress_clean['config_source'] = 'companies_config.py'

print(f"✅ Colonnes standardisées: {list(df_congress_clean.columns)}")
print(f"\n📋 Aperçu (from config tickers):")
cols_to_show = [col for col in ['politician_name', 'transaction_date', 'ticker', 'transaction_type', 'config_source'] if col in df_congress_clean.columns]
print(df_congress_clean[cols_to_show].head(5))

# Vérifier les types de transactions
print(f"\n📈 TYPES DE TRANSACTIONS (avant standardisation):")
if 'transaction_type' in df_congress_clean.columns:
    print(df_congress_clean['transaction_type'].value_counts())
    
# Mapper to BUY/SELL
print(f"\n🔄 MAPPING BUY/SELL (standardisation):")
buy_terms = ['Purchase', 'Buy']
sell_terms = ['Sale', 'Sell']

def normalize_transaction_type(val):
    if pd.isna(val):
        return val
    val_str = str(val).lower()
    if any(term.lower() in val_str for term in buy_terms):
        return 'BUY'
    elif any(term.lower() in val_str for term in sell_terms):
        return 'SELL'
    else:
        return val

df_congress_clean['transaction_type_normalized'] = df_congress_clean['transaction_type'].apply(normalize_transaction_type)

print("Avant mapping:", df_congress_clean['transaction_type'].unique()[:5])
print("Après mapping:", df_congress_clean['transaction_type_normalized'].unique())
print("\nComptage après mapping:")
print(df_congress_clean['transaction_type_normalized'].value_counts())

print(f"\n2️⃣ VALIDATION: Vérifier que tous tickers sont dans config")
unique_tickers_in_data = df_congress_clean['ticker'].unique()
missing_from_config = set(unique_tickers_in_data) - set(TICKERS)
if missing_from_config:
    print(f"⚠️ ERREUR: Tickers non trouvés dans config: {missing_from_config}")
else:
    print(f"✅ OK: Tous les tickers dans données sont dans companies_config.py")
    print(f"   Tickers dans données: {sorted(unique_tickers_in_data)}")


## 9️⃣ Statistiques Descriptives Complètes

In [ ]:
print("📊 STEP 8: RÉSUMÉ COMPLET DES DONNÉES - Config-Driven")
print("="*70)
print(f"📍 Source: companies_config.py ({len(TICKERS)} tickers)\n")

summary = {
    "source": {
        "config_file": "prod/config/companies_config.py",
        "tickers": TICKERS,
        "total_tickers": len(TICKERS),
    },
    "Congressional Trading": {
        "count": len(df_congress_filtered),
        "columns": list(df_congress_filtered.columns),
        "unique_tickers": df_congress_filtered['Ticker'].nunique() if 'Ticker' in df_congress_filtered.columns else 0,
        "tickers_in_data": sorted(df_congress_filtered['Ticker'].unique().tolist()) if 'Ticker' in df_congress_filtered.columns else [],
        "politicians": df_congress_filtered['Representative'].nunique() if 'Representative' in df_congress_filtered.columns else 'N/A',
    },
    "Senate Trading": {
        "count": len(df_senate_filtered) if df_senate_filtered is not None else 0,
        "columns": list(df_senate_filtered.columns) if df_senate_filtered is not None else [],
        "unique_tickers": df_senate_filtered['Ticker'].nunique() if df_senate_filtered is not None and 'Ticker' in df_senate_filtered.columns else 0,
    },
    "House Trading": {
        "count": len(df_house_filtered) if df_house_filtered is not None else 0,
        "columns": list(df_house_filtered.columns) if df_house_filtered is not None else [],
        "unique_tickers": df_house_filtered['Ticker'].nunique() if df_house_filtered is not None and 'Ticker' in df_house_filtered.columns else 0,
    },
    "Insider Trading": {
        "count": len(df_insiders_filtered) if df_insiders_filtered is not None else 0,
        "columns": list(df_insiders_filtered.columns) if df_insiders_filtered is not None else [],
        "unique_tickers": df_insiders_filtered['Ticker'].nunique() if df_insiders_filtered is not None and 'Ticker' in df_insiders_filtered.columns else 0,
    },
    "WSB Sentiment": {
        "count": len(df_wsb_filtered) if df_wsb_filtered is not None else 0,
        "columns": list(df_wsb_filtered.columns) if df_wsb_filtered is not None else [],
        "unique_tickers": df_wsb_filtered['Ticker'].nunique() if df_wsb_filtered is not None and 'Ticker' in df_wsb_filtered.columns else 0,
    }
}

# Display summary
print("🏢 CONFIGURATION (source of truth):")
print(f"   File: {summary['source']['config_file']}")
print(f"   Total tickers: {summary['source']['total_tickers']}")
print(f"   Tickers: {', '.join(summary['source']['tickers'][:5])}... ({len(TICKERS)} total)")

print("\n📊 DATA SUMMARY (filtrés par config):")
for source, info in summary.items():
    if source == 'source':
        continue
    print(f"\n   {source}:")
    for key, val in info.items():
        if key == 'columns':
            if val:
                print(f"      {key}: {', '.join(val[:3])}... ({len(val)} total)")
        elif key == 'tickers_in_data':
            print(f"      {key}: {val}")
        else:
            print(f"      {key}: {val}")

# Save summary to JSON
summary_path = Path("c:/n8n-local-stack/local_files/smart_money/quiverquant_data_summary.json")
summary_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)
    
print(f"\n✅ Résumé sauvegardé: {summary_path}")

print("\n" + "="*70)
print("✅ TOUTES LES DONNÉES SONT FILTRÉES PAR companies_config.py")
print("="*70)


## 🔟 Export & Intégration

In [ ]:
print("💾 STEP 9: EXPORT & INTÉGRATION - Config-Driven")
print("="*70)
print(f"📍 Source: companies_config.py ({len(TICKERS)} tickers)\n")

# Export Congressional cleaned (déjà filtré)
export_path = Path("c:/n8n-local-stack/local_files/smart_money/congressional_trades_explored.csv")
export_path.parent.mkdir(parents=True, exist_ok=True)

df_congress_clean.to_csv(export_path, index=False)
print(f"✅ Congressional trades exportés: {export_path}")
print(f"   Taille: {len(df_congress_clean)} lignes")
print(f"   Tickers: {len(df_congress_clean['ticker'].unique())} uniques (from config)")

# Export as JSON
json_path = Path("c:/n8n-local-stack/local_files/smart_money/congressional_trades_explored.json")
df_congress_clean.to_json(json_path, orient='records', date_format='iso', indent=2)
print(f"✅ JSON exporté: {json_path}")

# Config-driven export (metadata)
config_metadata = {
    "source_config": "prod/config/companies_config.py",
    "tickers_analyzed": TICKERS,
    "total_tickers": len(TICKERS),
    "data_types": {
        "congressional_trades": {
            "count": len(df_congress_clean),
            "tickers": sorted(df_congress_clean['ticker'].unique().tolist()),
            "file": str(export_path)
        },
        "insider_trades": {
            "count": len(df_insiders_filtered) if df_insiders_filtered is not None else 0,
            "tickers": sorted(df_insiders_filtered['Ticker'].unique().tolist()) if df_insiders_filtered is not None else [],
        },
        "wsb_sentiment": {
            "count": len(df_wsb_filtered) if df_wsb_filtered is not None else 0,
            "tickers": sorted(df_wsb_filtered['Ticker'].unique().tolist()) if df_wsb_filtered is not None else [],
        }
    },
    "export_timestamp": datetime.now().isoformat(),
    "analysis_date": "2026-01-06"
}

metadata_path = Path("c:/n8n-local-stack/local_files/smart_money/config_driven_analysis_metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(config_metadata, f, indent=2, default=str)
    
print(f"✅ Config metadata exporté: {metadata_path}")

# Résumé pour intégration
print("\n" + "="*70)
print("📝 RÉSUMÉ POUR INTÉGRATION SMART MONEY (CONFIG-DRIVEN)")
print("="*70)

print(f"""
✅ CONFIG SOURCE: prod/config/companies_config.py
   Total tickers: {len(TICKERS)}
   Tickers: {', '.join(TICKERS[:5])}... (15 total)

✅ DONNÉES DISPONIBLES (filtrées par config):
   1. Congressional Trading: {len(df_congress_clean)} trades
   2. Senate Trading: {len(df_senate_filtered) if df_senate_filtered is not None else 0} trades
   3. House Trading: {len(df_house_filtered) if df_house_filtered is not None else 0} trades
   4. Insider Trading: {len(df_insiders_filtered) if df_insiders_filtered is not None else 0} trades
   5. WallStreetBets Sentiment: {len(df_wsb_filtered) if df_wsb_filtered is not None else 0} mentions

📊 QUALITÉ DES DONNÉES:
   ✓ Congressional: Excellent (dates, noms, tickers, transactions)
   ✓ Standardisation: Possible (BUY/SELL mapping)
   ✓ Config alignment: 100% (tous filtrés par companies_config.py)
   ✓ Intégration: Prête pour Smart Money Tracker

🔄 COMMENT UTILISER:
   1. Importer companies_config.py en haut de chaque script
   2. Appeler get_all_companies() pour obtenir liste de tickers
   3. Filtrer df[df['Ticker'].isin(TICKERS)] à chaque étape
   4. Assurer que toutes les analyses utilisent la même config

💡 EXEMPLES:
   # ✅ BON: Config-driven
   from config.companies_config import get_all_companies
   companies = get_all_companies()
   TICKERS = [c['ticker'] for c in companies]
   df_filtered = df[df['Ticker'].isin(TICKERS)]
   
   # ❌ MAUVAIS: Hardcoded
   TICKERS = ['NVDA', 'MSFT', 'GOOGL', ...]  # Problème!

🔗 PROCHAINES ÉTAPES:
   1. Intégrer dans edgar_smart_money_analyzer.py (utiliser config)
   2. Créer signaux politiques (config-driven loop)
   3. Combiner avec Smart Money metrics (maintenir config source)
   4. Tester sur historical data (reproducible via config)

⚠️ NOTES IMPORTANTES:
   - Congressional data a des délais de report (5-45 jours)
   - Insider trades ont aussi des délais réglementaires
   - WSB sentiment est real-time mais bruyant
   - À combiner avec d'autres signaux pour meilleur signal
   - TOUJOURS commencer par companies_config.py comme source of truth
""")

print(f"\n✅ Notebook exploration complète et CONFIG-DRIVEN!")
print(f"📁 Fichiers générés:")
print(f"   - {export_path}")
print(f"   - {json_path}")
print(f"   - {metadata_path}")


## 🔍 BONUS: Limitations de l'API & Solutions

In [ ]:
print("📚 APPENDIX: Comment les Autres Scripts Utilisent la Config")
print("="*70)

print("""
🔍 SCAN DES SCRIPTS PRODUITS

══════════════════════════════════════════════════════════════════════

1️⃣ analyze_all_sentiment.py
─────────────────────────────
Location: prod/analysis/analyze_all_sentiment.py (line 18)

✅ CONFIG USAGE:
   from config.companies_config import get_all_companies
   
   def analyze_all_companies():
       companies = get_all_companies()
       
       # Filtrer les compagnies publiques (avec options)
       public_companies = [c for c in companies 
                          if c['ticker'] not in ['OPENAI', 'ANTHROPIC', 'COHERE', 'MISTRAL']]
       
       for i, company in enumerate(public_companies, 1):
           ticker = company['ticker']
           # Analyser chaque ticker

✅ PATTERN: Utilise get_all_companies(), filtrer publics vs privés, itérer

══════════════════════════════════════════════════════════════════════

2️⃣ collect_options.py
──────────────────────
Location: prod/collection/collect_options.py (line 32)

✅ CONFIG USAGE:
   from prod.config.companies_config import get_all_companies
   
   def collect_all_options():
       all_companies = get_all_companies()
       
       for ticker in [c['ticker'] for c in all_companies]:
           # Collecte données options pour ce ticker

✅ PATTERN: Import config, extract TICKERS, boucle sur chacun

══════════════════════════════════════════════════════════════════════

3️⃣ batch_loader_v2.py
──────────────────────
Location: prod/collection/batch_loader_v2.py (line 34)

✅ CONFIG USAGE:
   from config.companies_config import get_all_companies
   
   def load_batch():
       companies = get_all_companies()
       
       # Charger données batch pour tous les tickers

✅ PATTERN: Utilise companies_config pour définir scope du batch

══════════════════════════════════════════════════════════════════════

4️⃣ smart_money_analyzer.py
───────────────────────────
Location: prod/analysis/smart_money_analyzer.py (line 287-301)

✅ CONFIG USAGE:
   def analyze_smart_money(tickers_filter=None):
       '''
       Si tickers_filter is None:
           Utiliser companies_config comme filtre par défaut
       '''
       if tickers_filter is None:
           from prod.config.companies_config import COMPANIES_CONFIG
           tickers_filter = [c['ticker'] for c in COMPANIES_CONFIG]
       
       # Analyser uniquement les tickers configurés

✅ PATTERN: Config en fallback si pas de param, garantit cohérence

══════════════════════════════════════════════════════════════════════

5️⃣ merge_options_into_sentiment.py
──────────────────────────────────
Location: prod/analysis/merge_options_into_sentiment.py (line 113-115)

✅ CONFIG USAGE:
   print(f"\\n📋 Mode MANUAL: Utilisation de companies_config.py")
   
   if MANUAL_MODE:
       companies = get_all_companies()
       # Merge pour tous les tickers configurés

✅ PATTERN: Option explicite pour utiliser config (vs CSV)

══════════════════════════════════════════════════════════════════════

📊 PATTERN GÉNÉRAL
─────────────────

Tous les scripts suivent ce pattern:

   # 1. Import config
   from config.companies_config import get_all_companies
   
   # 2. Récupérer les tickers configurés
   companies = get_all_companies()
   tickers = [c['ticker'] for c in companies]
   
   # 3. Filtrer ou boucler sur ces tickers
   for ticker in tickers:
       process(ticker)
   
   # 4. Résultats s'alignent automatiquement avec config

✅ AVANTAGE: Si on change companies_config.py, TOUS les scripts changent!
   - Pas de hardcoded tickers
   - Source of truth unique
   - Easy maintenance

══════════════════════════════════════════════════════════════════════

🔴 ANTIPATTERNS À ÉVITER
──────────────────────

❌ MAUVAIS: Hardcoded tickers
   TICKERS = ['NVDA', 'MSFT', 'GOOGL', 'META', ...]  # Hard à maintenir!
   
❌ MAUVAIS: Pas de config usage
   if ticker in ['NVDA', 'MSFT']:  # Pas de cohérence
   
❌ MAUVAIS: Config ignorée dans une boucle
   for ticker in HARDCODED_LIST:  # Devrait être [c['ticker'] for c in get_all_companies()]

✅ BON: Toujours utiliser companies_config comme source
   companies = get_all_companies()  # Always start here
   for c in companies:
       process(c['ticker'])  # Uses config

══════════════════════════════════════════════════════════════════════
""")

print("\n✅ CONFIG ARCHITECTURE REVIEW COMPLETE")


⚠️ LIMITATIONS DE L'API QUIVERQUANT

❌ PROBLÈME #1: LIMITE DE 1000 RÉSULTATS
─────────────────────────────────────────
Raison: Protection contre surcharge API (pagination)
Conséquence: Seulement les 1000 trades RÉCENTS retournés

Analyse des dates:

   Date min:     2025-01-02
   Date max:     2025-12-16
   Span:         348 jours (1.0 ans)
   Trades/jour:  2.9


❌ PROBLÈME #2: DÉLAI DE REPORTING (1 AN)
──────────────────────────────────────────
Raison: Délais légaux avant disclosure public
- Congressional trades: 5 à 45 jours après transaction
- Insider trades: Jusqu'à 45 jours après transaction
- Résultat: Données actuelles = transactions de mois/années antérieurs!

Example du notebook:
   Transaction date: 2025-11-21
   Report date:      2025-12-29
   Délai:            38 jours ✅ Normal

❌ PROBLÈME #3: "1 AN" SIGNIFIE QUOI?
──────────────────────────────────────
C'est PAS une limite de l'API, c'est:
✓ Fenêtre temporelle des données disponibles
✓ 1000 résultats les plus récents = ~1 

## 💡 Implémentation: Solution de Caching Local

In [6]:
print("🔄 SOLUTION: ACCUMULATION PROGRESSIVE")
print("="*70)

# Simuler un cache accumulatif
cache_dir = Path("c:/n8n-local-stack/local_files/smart_money/quiverquant_cache")
cache_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Cache directory: {cache_dir}")

# Fonction pour charger ou créer cache
def load_or_create_cache(source_name, max_age_days=1):
    """
    Load cached data ou fetch new if too old
    """
    cache_file = cache_dir / f"{source_name}_cache.parquet"
    
    if cache_file.exists():
        # Check age
        mod_time = datetime.fromtimestamp(cache_file.stat().st_mtime)
        age = (datetime.now() - mod_time).days
        
        if age < max_age_days:
            print(f"✅ Cache valide ({age} jours old)")
            return pd.read_parquet(cache_file), True
        else:
            print(f"⚠️ Cache expiré ({age} jours old)")
            return None, False
    else:
        print(f"📝 Pas de cache - créer nouveau")
        return None, False

# Fonction pour sauvegarder avec historique
def save_with_history(df_new, source_name, max_keep=5):
    """
    Save new data and keep rotation of old caches
    """
    cache_file = cache_dir / f"{source_name}_cache.parquet"
    
    # Si cache existe, charger et merger
    if cache_file.exists():
        df_old = pd.read_parquet(cache_file)
        print(f"   Old cache: {len(df_old)} lignes")
        
        # Merger (éviter duplicates)
        df_merged = pd.concat([df_new, df_old]).drop_duplicates(
            subset=['Representative', 'TransactionDate', 'Ticker'] if 'Representative' in df_new.columns else None,
            keep='first'
        )
        print(f"   After merge: {len(df_merged)} lignes")
    else:
        df_merged = df_new
        
    # Save
    df_merged.to_parquet(cache_file)
    print(f"   ✅ Saved: {cache_file}")
    
    # Rotate old backups (keep last 5)
    backup_files = sorted(cache_dir.glob(f"{source_name}_backup_*.parquet"))
    for f in backup_files[:-max_keep]:
        f.unlink()
        
    return df_merged

# TEST
print("\n1️⃣ PREMIÈRE EXÉCUTION (pas de cache):")
df_cached, found = load_or_create_cache("congressional_trading")
if not found:
    df_cached = client.congress_trading()
    df_merged = save_with_history(df_cached, "congressional_trading")
    print(f"\n📊 Total historique: {len(df_merged)} trades")

print("\n2️⃣ DEUXIÈME EXÉCUTION (avec cache):")
df_cached2, found2 = load_or_create_cache("congressional_trading")
if found2:
    print(f"   Loaded: {len(df_cached2)} trades from cache")

print("\n" + "="*70)
print("✅ RÉSULTAT:")
print("""
Au lieu de:
   - Fetch 1000 aujourd'hui
   - Perte après 1 an (circular buffer)

Maintenant:
   - Fetch 1000 daily
   - Accumule historique localement
   - 1000 * 365 = 365,000 trades potentiels!
   - Croissance progressive sans perte
   
Avantage: Après 1 an de collection, avoir ~365K trades
         Puis utiliser /historical/{ticker} pour des tickers spécifiques
""")

print(f"\n📁 Cache saved at: {cache_dir}")

🔄 SOLUTION: ACCUMULATION PROGRESSIVE

📁 Cache directory: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache

1️⃣ PREMIÈRE EXÉCUTION (pas de cache):
📝 Pas de cache - créer nouveau
📡 Fetching: https://api.quiverquant.com/beta/live/congresstrading
   ✅ Saved: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache\congressional_trading_cache.parquet

📊 Total historique: 1000 trades

2️⃣ DEUXIÈME EXÉCUTION (avec cache):
✅ Cache valide (0 jours old)
   Loaded: 1000 trades from cache

✅ RÉSULTAT:

Au lieu de:
   - Fetch 1000 aujourd'hui
   - Perte après 1 an (circular buffer)

Maintenant:
   - Fetch 1000 daily
   - Accumule historique localement
   - 1000 * 365 = 365,000 trades potentiels!
   - Croissance progressive sans perte
   
Avantage: Après 1 an de collection, avoir ~365K trades
         Puis utiliser /historical/{ticker} pour des tickers spécifiques


📁 Cache saved at: c:\n8n-local-stack\local_files\smart_money\quiverquant_cache
